In [6]:
from torchvision import models
import torch
from torch import nn
from torch import optim
import torch.nn.functional as F
from torchvision import datasets, transforms

In [2]:
model = models.resnet50(pretrained=True)

Downloading: "https://download.pytorch.org/models/resnet50-19c8e357.pth" to /home/jovyan/.torch/models/resnet50-19c8e357.pth
102502400.0 bytes


In [7]:
model

ResNet(
  (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): Bottleneck(
      (conv1): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv3): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn3): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace)
      (downsample): Sequential(
        (0): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1), bias=F

In [9]:
device = torch.device('cuda' if torch.cuda.is_available else 'cpu')

In [12]:
for p in model.parameters():
    p.requires_grad = False

classifier = nn.Sequential(nn.Linear(2048, 512),
                          nn.ReLU(),
                          nn.Dropout(p=0.2),
                          nn.Linear(512, 3),
                          nn.LogSoftmax(dim=1))
model.fc = classifier
criterion = nn.NLLLoss()
optimizer = optim.Adam(model.fc.parameters(), lr=0.003)


In [14]:
train_path = './idcard_bg'
transform = transforms.Compose([transforms.Resize(255),
                               transforms.CenterCrop(224),
                               transforms.ToTensor()])
dataset = datasets.ImageFolder(train_path, transform=transform)
dataloader = torch.utils.data.DataLoader(dataset, batch_size=4, shuffle=True)

In [18]:
epochs = 10
steps = 0

print('begin to train...')
for epoch in range(epochs):
    train_loss = 0
    for images, labels in dataloader:
        steps += 1
        optimizer.zero_grad()
        output = model(images)
        loss = criterion(output, labels)
        loss.backward()
        optimizer.step()
        
        train_loss += loss.item()
        
    print(f"epoch: {epoch}, loss: {train_loss}")
        

begin to tain...
epoch: 0, loss: 9.125800728797913
epoch: 1, loss: 10.949593961238861
epoch: 2, loss: 18.081299364566803
epoch: 3, loss: 10.02782341837883
epoch: 4, loss: 5.933320760726929
epoch: 5, loss: 13.28603084385395
epoch: 6, loss: 10.501172870397568
epoch: 7, loss: 4.104996383190155
epoch: 8, loss: 7.565685838460922
epoch: 9, loss: 6.801829323172569
